# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Big Data Real-Time Analytics com Python e Spark</font>

# <font color='blue'>Capítulo 10</font>

### *********** Atenção: *********** 
Utilize Java JDK 1.8 ou 11 e Apache Spark 2.4.2

****** Caso receba mensagem de erro "name 'sc' is not defined", interrompa o pyspark e apague o diretório metastore_db no mesmo diretório onde está este Jupyter notebook ******

Acesse http://localhost:4040 sempre que quiser acompanhar a execução dos jobs

# Spark Streaming - Twitter

#### Vamos por meio desse projeto ler em tempo real os tweets feitos em nossa conta do twitter e aplicar em tempo real um método de classificação de machine learning ( Naive Bayes ), para classificar se o Tweet feito corresponde a um sentimento Positivo ou Negativo 

### 1 - Importando as ferramentas que usaremos e Criando as Sessões necessárias no Spark

In [ ]:
# Pode ser necessário instalar esses pacotes
#!pip install requests_oauthlib
#!pip install twython
#!pip install nltk

In [ ]:
# Módulos usados
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
#Autenticação de sessão para poder acessar o twitter
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json

In [ ]:
# Pacote NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [ ]:
# Frequência de update
INTERVALO_BATCH = 5

In [4]:
# Criando o StreamingContext
ssc = StreamingContext(sc, INTERVALO_BATCH)

### 2 - Treinando o Classificador de Análise de Sentimento

#### Vamos utilizar o seguinte dataset para treinar o nosso modelo de classificação

Uma parte essencial da criação de um algoritmo de análise de sentimento (ou qualquer algoritmo de mineração de dados) é ter um conjunto de dados abrangente ou "Corpus" para o aprendizado, bem como um conjunto de dados de teste para garantir que a precisão do seu algoritmo atende aos padrões que você espera. Isso também permitirá que você ajuste o seu algoritmo a fim de deduzir melhores (ou mais precisas) características de linguagem natural que você poderia extrair do texto e que vão contribuir para a classificação de sentimento, em vez de usar uma abordagem genérica. Tomaremos como base o dataset de treino fornecido pela Universidade de Michigan, para competições do Kaggle --> https://inclass.kaggle.com/c/si650winter11.

Esse dataset contém 1,578,627 tweets classificados e cada linha é marcada como: 

### 1 para o sentimento positivo 
### 0 para o sentimento negativo 

In [5]:
# Lendo o arquivo texto e criando um RDD em memória com Spark
arquivo = sc.textFile("dataset/dataset_analise_sentimento.csv")

In [6]:
type(arquivo)

pyspark.rdd.RDD

In [7]:
arquivo.take(10)

['ItemID,Sentiment,SentimentSource,SentimentText',
 '1,0,Sentiment140,                     is so sad for my APL friend.............',
 '2,0,Sentiment140,                   I missed the New Moon trailer...',
 '3,1,Sentiment140,              omg its already 7:30 :O',
 "4,0,Sentiment140,          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...",
 '5,0,Sentiment140,         i think mi bf is cheating on me!!!       T_T',
 '6,0,Sentiment140,         or i just worry too much?        ',
 '7,1,Sentiment140,       Juuuuuuuuuuuuuuuuussssst Chillin!!',
 '8,0,Sentiment140,       Sunny Again        Work Tomorrow  :-|       TV Tonight',
 '9,1,Sentiment140,      handed in my uniform today . i miss you already']

#### Podemos observar que cada avaliação está separada por uma virgula e que no RDD  possui cabeçalho
Podemos também observar que a classificação de cada frase está na posição 1 

In [8]:
# Removendo o cabeçalho

# Informando que o cabeçalho está na primeira posição do RDD
header = arquivo.take(1)[0]
#Salvando o novo dataset Sem o cabeçalho 
dataset = arquivo.filter(lambda line: line != header)

In [9]:
type(dataset)

pyspark.rdd.PipelinedRDD

In [10]:
# Essa função vai fazer o primerio pré processamento dados da RDD

#1°- Ele mapeia cada uma das linhas, ou seja, cada uma das avaliações, e divide elas em colunas,
# assim estamos dividindo as frases em palavras

# 2° Cria uma tupla de valores em que o primeiro é exatamente um array de palavras das frases e o segundo
# valor é a classificação dessa frase se ela é classe 1 ( positiva) ou classe 0 (negativa)

# 3° Por fim como estamos trabalhando com frases em um algoritmo de classificação temos uma importante etapa
# dessa função que é retirar todo tipo de pontuação das frases e substituir por espaços em Branco 
def get_row(line):
  row = line.split(',')
  sentimento = row[1]
  tweet = row[3].strip()
  translator = str.maketrans({key: None for key in string.punctuation})
  tweet = tweet.translate(translator)
  tweet = tweet.split(' ')
  tweet_lower = []
  for word in tweet:
    tweet_lower.append(word.lower())
  return (tweet_lower, sentimento)

In [11]:
# Aplica a função a cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line))

In [12]:
dataset_treino.take(50)

[(['is', 'so', 'sad', 'for', 'my', 'apl', 'friend'], '0'),
 (['i', 'missed', 'the', 'new', 'moon', 'trailer'], '0'),
 (['omg', 'its', 'already', '730', 'o'], '1'),
 (['',
   'omgaga',
   'im',
   'sooo',
   '',
   'im',
   'gunna',
   'cry',
   'ive',
   'been',
   'at',
   'this',
   'dentist',
   'since',
   '11',
   'i',
   'was',
   'suposed',
   '2',
   'just',
   'get',
   'a',
   'crown',
   'put',
   'on',
   '30mins'],
  '0'),
 (['i',
   'think',
   'mi',
   'bf',
   'is',
   'cheating',
   'on',
   'me',
   '',
   '',
   '',
   '',
   '',
   '',
   'tt'],
  '0'),
 (['or', 'i', 'just', 'worry', 'too', 'much'], '0'),
 (['juuuuuuuuuuuuuuuuussssst', 'chillin'], '1'),
 (['sunny',
   'again',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   'work',
   'tomorrow',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   'tv',
   'tonight'],
  '0'),
 (['handed',
   'in',
   'my',
   'uniform',
   'today',
   '',
   'i',
   'miss',
   'you',
   'already'],
  '1'),
 (['hmmmm', 'i'

In [13]:
# Cria um objeto SentimentAnalyzer do pacote NLTK
sentiment_analyzer = SentimentAnalyzer()

#### Vamos usar o NLTK para remover as StopWords, The , a , ....

Usando as stopwords do pacote NLTK

In [14]:
# Certifique-se de ter espaço em disco - Aproximadamente 5GB
# https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
# nltk.download()
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/bulka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from IPython.display import Image
Image(url = 'ntlkdata.png')

In [16]:
# Obtém a lista de stopwords em Inglês e as coloca na lista Stopwords_all
stopwords_all = []
for word in stopwords.words('english'):
  stopwords_all.append(word)
  stopwords_all.append(word + '_NEG')

In [17]:
stopwords_all

['i',
 'i_NEG',
 'me',
 'me_NEG',
 'my',
 'my_NEG',
 'myself',
 'myself_NEG',
 'we',
 'we_NEG',
 'our',
 'our_NEG',
 'ours',
 'ours_NEG',
 'ourselves',
 'ourselves_NEG',
 'you',
 'you_NEG',
 "you're",
 "you're_NEG",
 "you've",
 "you've_NEG",
 "you'll",
 "you'll_NEG",
 "you'd",
 "you'd_NEG",
 'your',
 'your_NEG',
 'yours',
 'yours_NEG',
 'yourself',
 'yourself_NEG',
 'yourselves',
 'yourselves_NEG',
 'he',
 'he_NEG',
 'him',
 'him_NEG',
 'his',
 'his_NEG',
 'himself',
 'himself_NEG',
 'she',
 'she_NEG',
 "she's",
 "she's_NEG",
 'her',
 'her_NEG',
 'hers',
 'hers_NEG',
 'herself',
 'herself_NEG',
 'it',
 'it_NEG',
 "it's",
 "it's_NEG",
 'its',
 'its_NEG',
 'itself',
 'itself_NEG',
 'they',
 'they_NEG',
 'them',
 'them_NEG',
 'their',
 'their_NEG',
 'theirs',
 'theirs_NEG',
 'themselves',
 'themselves_NEG',
 'what',
 'what_NEG',
 'which',
 'which_NEG',
 'who',
 'who_NEG',
 'whom',
 'whom_NEG',
 'this',
 'this_NEG',
 'that',
 'that_NEG',
 "that'll",
 "that'll_NEG",
 'these',
 'these_NEG',


In [18]:
# Obtém 10.000 tweets do dataset de treino
dataset_treino_amostra = dataset_treino.take(10000)

In [19]:
# Vamos retornar todas as palavras dessas 10000 linhas que não são stopwords
# Usando o sentiment analyser do NTlK e sua função All words 
all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [20]:
all_words_neg

['is',
 'so',
 'sad',
 'for',
 'my',
 'apl',
 'friend',
 'i',
 'missed',
 'the',
 'new',
 'moon',
 'trailer',
 'omg',
 'its',
 'already',
 '730',
 'o',
 '',
 'omgaga',
 'im',
 'sooo',
 '',
 'im',
 'gunna',
 'cry',
 'ive',
 'been',
 'at',
 'this',
 'dentist',
 'since',
 '11',
 'i',
 'was',
 'suposed',
 '2',
 'just',
 'get',
 'a',
 'crown',
 'put',
 'on',
 '30mins',
 'i',
 'think',
 'mi',
 'bf',
 'is',
 'cheating',
 'on',
 'me',
 '',
 '',
 '',
 '',
 '',
 '',
 'tt',
 'or',
 'i',
 'just',
 'worry',
 'too',
 'much',
 'juuuuuuuuuuuuuuuuussssst',
 'chillin',
 'sunny',
 'again',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'work',
 'tomorrow',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'tv',
 'tonight',
 'handed',
 'in',
 'my',
 'uniform',
 'today',
 '',
 'i',
 'miss',
 'you',
 'already',
 'hmmmm',
 'i',
 'wonder',
 'how',
 'she',
 'my',
 'number',
 '',
 'i',
 'must',
 'think',
 'about',
 'positive',
 'thanks',
 'to',
 'all',
 'the',
 'haters',
 'up',
 'in',
 'my',
 'face',
 'all',
 'day',
 '112102',
 't

In [21]:
all_words_neg_nostops

['sad',
 'apl',
 'friend',
 'missed',
 'new',
 'moon',
 'trailer',
 'omg',
 'already',
 '730',
 '',
 'omgaga',
 'im',
 'sooo',
 '',
 'im',
 'gunna',
 'cry',
 'ive',
 'dentist',
 'since',
 '11',
 'suposed',
 '2',
 'get',
 'crown',
 'put',
 '30mins',
 'think',
 'mi',
 'bf',
 'cheating',
 '',
 '',
 '',
 '',
 '',
 '',
 'tt',
 'worry',
 'much',
 'juuuuuuuuuuuuuuuuussssst',
 'chillin',
 'sunny',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'work',
 'tomorrow',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'tv',
 'tonight',
 'handed',
 'uniform',
 'today',
 '',
 'miss',
 'already',
 'hmmmm',
 'wonder',
 'number',
 '',
 'must',
 'think',
 'positive',
 'thanks',
 'haters',
 'face',
 'day',
 '112102',
 'weekend',
 'sucked',
 'far',
 'jb',
 'isnt',
 'showing_NEG',
 'australia_NEG',
 'ok',
 'thats',
 'win',
 'lt',
 'way',
 'feel',
 'right',
 '',
 '',
 '',
 '',
 'awhhe',
 'man',
 'im',
 'completely',
 'useless',
 'rt',
 'funny',
 'feeling',
 'strangely',
 'fine',
 'im',
 'gonna',
 'go',
 'listen',
 'semisonic',


In [22]:
# Cria um unigram (n-grama) e extrai as features
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops, top_n = 200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

In [23]:
type(training_set)

nltk.collections.LazyMap

In [24]:
print(training_set)

[({'contains()': False, 'contains(im)': False, 'contains(_NEG)': False, 'contains(followfriday)': False, 'contains(amp)': False, 'contains(dont)': False, 'contains(day)': False, 'contains(love)': False, 'contains(like)': False, 'contains(cant)': False, 'contains(good)': False, 'contains(get)': False, 'contains(go)': False, 'contains(today)': False, 'contains(got)': False, 'contains(want)': False, 'contains(time)': False, 'contains(going)': False, 'contains(back)': False, 'contains(one)': False, 'contains(sad)': True, 'contains(really)': False, 'contains(miss)': False, 'contains(u)': False, 'contains(work)': False, 'contains(new)': False, 'contains(2)': False, 'contains(last)': False, 'contains(still)': False, 'contains(twitter)': False, 'contains(night)': False, 'contains(great)': False, 'contains(lol)': False, 'contains(follow)': False, 'contains(need)': False, 'contains(see)': False, 'contains(much)': False, 'contains(myweakness)': False, 'contains(get_NEG)': False, 'contains(didnt)'

In [25]:
# Treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)

Training classifier


In [26]:
# Testa o classificador em algumas sentenças
test_sentence1 = [(['this', 'program', 'is', 'bad'], '')]
test_sentence2 = [(['tough', 'day', 'at', 'work', 'today'], '')]
test_sentence3 = [(['good', 'wonderful', 'amazing', 'awesome'], '')]
test_set = sentiment_analyzer.apply_features(test_sentence1)
test_set2 = sentiment_analyzer.apply_features(test_sentence2)
test_set3 = sentiment_analyzer.apply_features(test_sentence3)

#### Com o Classificador pronto vamos preparar a conexão com o Twitter para aplicar esse classificador em tempo real 

In [27]:
# Autenticação do Twitter com os dados da nossa conta de desenvolvedor 
consumer_key = "xxx"
consumer_secret = "xxx"
access_token = "xxx"
access_token_secret = "xxx"

In [28]:
# Especifica a URL termo de busca
#Vamos buscar dados sobre o trump
search_term = 'Bolsonaro'
#Vamos buscar no próprio streaming do twitter 
sample_url = 'https://stream.twitter.com/1.1/statuses/sample.json'
#Filtrando os termos que tem a palavra trump
filter_url = 'https://stream.twitter.com/1.1/statuses/filter.json?track='+search_term

In [29]:
# Criando o objeto de atutenticação para o Twitter
auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_token, access_token_secret)

In [30]:
# Configurando o Stream
#Criando um RDD vazio
rdd = ssc.sparkContext.parallelize([0])
# Os dados vão chegando em tempo real assim precisamos criar a seguinte fila de RDDs para receber os dados 
stream = ssc.queueStream([], default = rdd)

In [31]:
type(stream)

pyspark.streaming.dstream.DStream

In [32]:
# Total de tweets por update
NUM_TWEETS = 500  

In [33]:
# Essa função conecta ao Twitter e retorna um número específico de Tweets (NUM_TWEETS)
def tfunc(t, rdd):
  return rdd.flatMap(lambda x: stream_twitter_data())

#Função que conecta notwitter e passa o filtro do Trump que estabelecemos 
def stream_twitter_data():
  response = requests.get(filter_url, auth = auth, stream = True)
  print(filter_url, response)
  count = 0
  for line in response.iter_lines():
    try:
      if count > NUM_TWEETS:
        break
      post = json.loads(line.decode('utf-8'))
      contents = [post['text']]
      count += 1
      yield str(contents)
    except:
      result = False

In [34]:
stream = stream.transform(tfunc)

In [35]:
#Retornando caa twitter da mesma forma que estamos coletando ou seja na forma literal 
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [36]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
  sentence = [(tweet, '')]
  test_set = sentiment_analyzer.apply_features(sentence)
  print(tweet, classifier.classify(test_set[0][0]))
  return(tweet, classifier.classify(test_set[0][0]))

In [37]:
# Essa função retorna o texto do Twitter
def get_tweet_text(rdd):
  for line in rdd:
    tweet = line.strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
      tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

In [38]:
# Cria uma lista vazia para os resultados
resultados = []

In [39]:
# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
  global resultados
  pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
  counts = pairs.reduceByKey(add)
  output = []
  for count in counts.collect():
    output.append(count)
  result = [time.strftime("%I:%M:%S"), output]
  resultados.append(result)
  print(result)

In [40]:
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [41]:
# Start streaming
ssc.start()
# ssc.awaitTermination()

In [42]:
cont = True
while cont:
  if len(resultados) > 5:
    cont = False

['04:18:28', []]
['04:18:33', []]
['04:18:37', []]
['04:18:42', []]
['04:18:47', []]
['04:18:52', []]


In [43]:
# Grava os resultados
rdd_save = ''+time.strftime("%I%M%S")
resultados_rdd = sc.parallelize(resultados)
resultados_rdd.saveAsTextFile(rdd_save)

In [44]:
# Visualiza os resultados
resultados_rdd.collect()

[['04:18:28', []],
 ['04:18:33', []],
 ['04:18:37', []],
 ['04:18:42', []],
 ['04:18:47', []],
 ['04:18:52', []]]

In [45]:
# Finaliza o streaming
ssc.stop()

# Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>